In [28]:
import pandas as pd
from sklearn.cross_validation import train_test_split

In [12]:
dat = pd.read_csv("../data/iris/datatest.txt", header=-1, 
                  names=['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
dat.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [18]:
dat['class'].value_counts()

Iris-virginica     50
Iris-versicolor    50
Iris-setosa        50
Name: class, dtype: int64

In [39]:
train, test = train_test_split(dat)

In [42]:
train.head()

,sepal length,sepal width,petal length,petal width,class
70,5.9,3.2,4.8,1.8,Iris-versicolor
89,5.5,2.5,4.0,1.3,Iris-versicolor
138,6.0,3.0,4.8,1.8,Iris-virginica
72,6.3,2.5,4.9,1.5,Iris-versicolor
38,4.4,3.0,1.3,0.2,Iris-setosa


In [43]:
test.head()

,sepal length,sepal width,petal length,petal width,class
82,5.8,2.7,3.9,1.2,Iris-versicolor
130,7.4,2.8,6.1,1.9,Iris-virginica
55,5.7,2.8,4.5,1.3,Iris-versicolor
4,5.0,3.6,1.4,0.2,Iris-setosa
131,7.9,3.8,6.4,2.0,Iris-virginica
